### User Input

In [ ]:
# MODEL PARAMETERS
model_settings ={
    'layers': 3,
    'dropout': 0.1,
    'padding': 'same',
    'n_models': 16,
    'trial': 1
}

### Path Setup

In [ ]:

layer_count = str(model_settings['layers'])
models_count = str(model_settings['n_models'])
trial_count = str(model_settings['trial'])

mfccs_json_path = "../mfccs_cnn_humpbackwhale_walrus_bowheadwhale_fin_finbackwhale_killerwhale_emptyocean.json"

csv_file_name = f'../model-stats/layers-{layer_count}_filters-1-{models_count}_n-trail{trial_count}.csv'

plot_file_name = f'../images/layers-{layer_count}_filters-1-{models_count}_n-trail{trial_count}.png'

### Imports

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np
import warnings

### Load MFCC Dataset

In [ ]:
from cnn_comparison_lib import load_cnn_json

X, y, L = load_cnn_json(mfccs_json_path)
print(f"mapping the marine mammals: {L}")

In [ ]:
# Check class counts and that labels are numerical order
np.unique(y, return_counts=True)

### Create Training, Validation, and Test Sets

In [ ]:
# create train, validation and test sets
from cnn_comparison_lib import prepare_datasets

X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(X, y, 0.6, 0.5) # test size, vailidation size
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])

### Build the CNN Models

In [ ]:
from cnn_comparison_lib import create_layered_cnn

model_list = []
for i in range(model_settings['n_models']):
    output_shape = i + 1
    model = create_layered_cnn(model_settings['layers'], input_shape, output_shape, model_settings)
    model_list.append(model)

### Get Trainable Parameter Count

In [ ]:
from cnn_comparison_lib import count_trainable_parameters

parameter_count = []
for model in model_list:
    count = count_trainable_parameters(model)
    parameter_count.append(count)

print(parameter_count)

### Train the Models

In [ ]:
from cnn_comparison_lib import plot_history

model_history = []
for model in model_list:
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=16, epochs=5)
    model_history.append(history)

### Evaluate the Models

In [ ]:
# evaluate the cnn on the test set
model_accuracy = []
for model in model_list:
    test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
    model_accuracy.append(test_accuracy)

### Plot the Models

In [ ]:
x = parameter_count
y = model_accuracy
plt.plot(x, y, 'o', color='black')
plt.xlabel("Number of Parameters")
plt.ylabel("Model Accuracy")
plt.savefig(plot_file_name)

### Save the Models

In [ ]:
saved_models_path = "../saved_model/layers" + str(model_settings['layers']) + "/paramcount"
for n_params, model in zip(parameter_count, model_list):
    model_name = saved_models_path + str(n_params) + '.h5'
    model.save(model_name)

### Save the Model Parameters and Accuracy Values

In [ ]:
import csv
from itertools import zip_longest

list1 = parameter_count
list2 = model_accuracy
d = [list1, list2]
export_data = zip_longest(*d, fillvalue = '')
with open(csv_file_name, 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("Parameter Count", "Baseline Accuracy"))
      wr.writerows(export_data)
myfile.close()